# How to style images using CycleGAN

In [1]:
from zenml.core.datasources.image_datasource import ImageDatasource
from zenml.core.pipelines.training_pipeline import TrainingPipeline
from zenml.core.steps.split.categorical_domain_split_step import \
    CategoricalDomainSplit
from examples.gan.trainer.trainer_step import CycleGANTrainer
from examples.gan.preprocessing.preprocessor import GANPreprocessor

First we instantiate our current ZenML repository. That way, if you run this notebook
multiple times in succession, you will not run into trouble with errors on datasource
creation.

In [2]:
# First initialize the repository by calling `zenml init` in this folder from the
# command line

#repo = Repository.get_instance()

### Creating the pipeline

Now we create the training pipeline that we will run our experiment with. We enable
caching by default, as it saves computation time by saving results that do not have to
be recomputed over and over again, such as splits or preprocessing.

In [3]:
gan_pipeline = TrainingPipeline(name="gan_test", enable_cache=True)

TypeError: Expected binary or unicode string, got PosixPath('/Users/nicholasjunge/workspaces/maiot/zenml')

### Creating the datasource

Next, we create the image datasource for the GAN. If you have not run it before, run the
`gan_images_ce.py` script located in this folder before executing the next cell,
which will create a base directory and a label file for use in your ZenML image source.

In [ ]:
try:
    ds = ImageDatasource(name="gan_images",
                         base_path="/Users/nicholasjunge/workspaces/maiot/ce_project/images_mini")
except Exception as e:
    print(e)
    #ds = repo.get_datasource_by_name('gan_images')

### Add a split

To prepare our images, we need to separate the real images from the styled ones (Monet
paintings in this case). Therefore, we add the image type (real or Monet) as a label
and split on that. The real images are going to be saved as evaluation data, because we
will use them later in image style generation.

In [ ]:
gan_pipeline.add_split(CategoricalDomainSplit(categorical_column="label",
                                              split_map={"train": ["monet"],
                                                         "eval": ["real"]}))

### Add a preprocessing step

Next, we add a small preprocessing step for our image. It is defined in the
`preprocessing.py` file in this folder, and contains two separate steps: Loading the
image (up to this point, it is persisted as a binary string) and normalizing its values
 between -1 and 1.

In [ ]:
gan_pipeline.add_preprocesser(GANPreprocessor())

### Add a training step

Now we come to what is the most involved part of this example - a custom trainer step.
It is defined in the `gan_functions.py` file in this folder, and contains a Keras
implementation of the CycleGAN model architecture. Along with this, it also comes with
utilities for loading data, preparing it as `tf.data.Dataset`s and signature definitions,
for when you want to push a model e.g. to Google Cloud AI Platform.

In [ ]:
gan_pipeline.add_trainer(CycleGANTrainer(epochs=25))

And that is it! If you studied the contents of the `gan_functions.py` file, you will
notice that there's also a custom callback function implemented. It uses TensorBoard
to display the same single, Monet-styled image generated by the CycleGAN after each
epoch of learning. There is a neat little slider that lets you look through all logged
epochs - that way, you can visualize the CycleGAN's style learning process.

That's it for this tutorial! By tuning some hyperparameters and training for more
epochs, you can also create high quality Monet renderings of real images. And when
you are successful, you immediately know the best configuration because it is all
logged by the immutability of the pipeline! That way, you can immediately share your
configuration for other people to reproduce.

We hope you had fun with this tutorial, and see you for the next one!